In [1]:
import pandas as pd
import numpy as np
import datetime
from dateutil.relativedelta import *

In [2]:
form13f = pd.read_csv("../RebellionData/rebellion_databases/form13f.csv", sep=";")

In [3]:
form13f["iFILING_DATE"]= pd.to_datetime(form13f["iFILING_DATE"]).dt.date
form13f["iPERIOD_END"]= pd.to_datetime(form13f["iPERIOD_END"]).dt.date

In [4]:
form13f

,iRECORD_ID,iCIK,iCUSIP,iPERIOD_END,iFILING_DATE,iAMEND,iRESTATEMENT,iTYPE,iQTY,iMARKET_VALUE,iLONG_FRACTION
0,4443748,216235,055622104,1999-09-30,2000-01-03,0,0,0,28287.0,3134550.0,0.009282
1,4443749,216235,020039103,1999-09-30,2000-01-03,0,0,0,79199.0,5690620.0,0.016851
2,4443750,216235,713448108,1999-09-30,2000-01-03,0,0,0,186140.0,5795950.0,0.017163
3,4443751,216235,002824100,1999-09-30,2000-01-03,0,0,0,169555.0,6290420.0,0.018627
4,4443752,216235,05964H105,1999-09-30,2000-01-03,0,0,0,16897.0,174250.0,0.000516
...,...,...,...,...,...,...,...,...,...,...,...
57569231,74561590,1730479,039483102,2020-03-31,2020-04-23,0,0,0,300.0,11000.0,0.000086
57569232,74561591,1730479,88160R101,2020-03-31,2020-04-23,0,0,0,25.0,13000.0,0.000102
57569233,74561592,1730479,921937793,2020-03-31,2020-04-23,0,0,0,230.0,25000.0,0.000196
57569234,74561593,1730479,863667101,2020-03-31,2020-04-23,0,0,0,1030.0,171000.0,0.001341


# Set the current (training) period: CHANGE THIS !!!

In [5]:
PERIOD_TRAIN = datetime.date(2018,6,30)
PERIOD_TRAIN

datetime.date(2018, 6, 30)

# I. Data Processing

## Filter individual records

1. Remove records that were filed before June 30, 2013 with iPERIOD_END later than PERIOD_TRAIN

In [6]:
form13f_2013 = form13f[(form13f.iFILING_DATE >= datetime.date(2013, 6, 30)) & (form13f.iPERIOD_END <= PERIOD_TRAIN)]
form13f_2013

,iRECORD_ID,iCIK,iCUSIP,iPERIOD_END,iFILING_DATE,iAMEND,iRESTATEMENT,iTYPE,iQTY,iMARKET_VALUE,iLONG_FRACTION
26088669,38587073,1349353,002824100,2013-06-30,2013-08-01,0,0,0,96510.0,3366000.0,0.023443
26088670,38587074,1349353,025816109,2013-06-30,2013-08-01,0,0,0,70214.0,5249000.0,0.036557
26088671,38587075,1349353,026874784,2013-06-30,2013-08-01,0,0,0,346115.0,15471000.0,0.107748
26088672,38587076,1349353,060505104,2013-06-30,2013-08-01,0,0,0,882340.0,11347000.0,0.079026
26088673,38587077,1349353,064058100,2013-06-30,2013-08-01,0,0,0,129065.0,3620000.0,0.025212
...,...,...,...,...,...,...,...,...,...,...,...
57510142,74502501,948046,46429B598,2018-06-30,2020-04-21,1,0,0,147975.0,4926000.0,0.000000
57510143,74502502,948046,141633107,2018-06-30,2020-04-21,1,0,0,186939.0,3903000.0,0.000000
57510144,74502503,948046,33740F888,2018-06-30,2020-04-21,1,0,0,400000.0,10012000.0,0.000000
57510145,74502504,948046,398905109,2018-06-30,2020-04-21,1,0,0,173751.0,10946000.0,0.000000


2. Remove records that are duplicates.
3. Remove records wherein QTY is zero.
4. Remove records wherein MARKET_VALUE is zero.

In [7]:
form13f_duplicate = form13f_2013.drop_duplicates()
form13f_QTY = form13f_duplicate[form13f_duplicate["iQTY"] != 0]
form13f_MARKET_VALUE = form13f_QTY[form13f_QTY["iMARKET_VALUE"] != 0]
form13f_MARKET_VALUE["FILING_INTERVAL"] = form13f_MARKET_VALUE["iFILING_DATE"]-form13f_MARKET_VALUE["iPERIOD_END"]
form13f_MARKET_VALUE

/var/folders/gd/f9gfs56x5qgb9q0h_h7dwwrc0000gn/T/ipykernel_4899/4067883774.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  form13f_MARKET_VALUE["FILING_INTERVAL"] = form13f_MARKET_VALUE["iFILING_DATE"]-form13f_MARKET_VALUE["iPERIOD_END"]


,iRECORD_ID,iCIK,iCUSIP,iPERIOD_END,iFILING_DATE,iAMEND,iRESTATEMENT,iTYPE,iQTY,iMARKET_VALUE,iLONG_FRACTION,FILING_INTERVAL
26088669,38587073,1349353,002824100,2013-06-30,2013-08-01,0,0,0,96510.0,3366000.0,0.023443,32 days
26088670,38587074,1349353,025816109,2013-06-30,2013-08-01,0,0,0,70214.0,5249000.0,0.036557,32 days
26088671,38587075,1349353,026874784,2013-06-30,2013-08-01,0,0,0,346115.0,15471000.0,0.107748,32 days
26088672,38587076,1349353,060505104,2013-06-30,2013-08-01,0,0,0,882340.0,11347000.0,0.079026,32 days
26088673,38587077,1349353,064058100,2013-06-30,2013-08-01,0,0,0,129065.0,3620000.0,0.025212,32 days
...,...,...,...,...,...,...,...,...,...,...,...,...
57510142,74502501,948046,46429B598,2018-06-30,2020-04-21,1,0,0,147975.0,4926000.0,0.000000,661 days
57510143,74502502,948046,141633107,2018-06-30,2020-04-21,1,0,0,186939.0,3903000.0,0.000000,661 days
57510144,74502503,948046,33740F888,2018-06-30,2020-04-21,1,0,0,400000.0,10012000.0,0.000000,661 days
57510145,74502504,948046,398905109,2018-06-30,2020-04-21,1,0,0,173751.0,10946000.0,0.000000,661 days


## Filter entire funds for each period

1. Funds must complete filing within 46 days of the end of the quarter.

In [8]:
# iCIK is fund identifier
form13f_46 = form13f_MARKET_VALUE.groupby(["iCIK", "iPERIOD_END"]).filter(lambda x:(x.FILING_INTERVAL <= datetime.timedelta(days = 45)).all())
form13f_46

,iRECORD_ID,iCIK,iCUSIP,iPERIOD_END,iFILING_DATE,iAMEND,iRESTATEMENT,iTYPE,iQTY,iMARKET_VALUE,iLONG_FRACTION,FILING_INTERVAL
26088669,38587073,1349353,002824100,2013-06-30,2013-08-01,0,0,0,96510.0,3366000.0,0.023443,32 days
26088670,38587074,1349353,025816109,2013-06-30,2013-08-01,0,0,0,70214.0,5249000.0,0.036557,32 days
26088671,38587075,1349353,026874784,2013-06-30,2013-08-01,0,0,0,346115.0,15471000.0,0.107748,32 days
26088672,38587076,1349353,060505104,2013-06-30,2013-08-01,0,0,0,882340.0,11347000.0,0.079026,32 days
26088673,38587077,1349353,064058100,2013-06-30,2013-08-01,0,0,0,129065.0,3620000.0,0.025212,32 days
...,...,...,...,...,...,...,...,...,...,...,...,...
50110418,67098553,1641864,422806208,2018-06-30,2018-08-13,0,0,0,531120.0,32372000.0,0.036413,44 days
50110419,67098554,1641864,143130102,2018-06-30,2018-08-13,0,0,0,1269870.0,92536000.0,0.104088,44 days
50110420,67098555,1641864,33829M101,2018-06-30,2018-08-13,0,0,0,11009.0,1076000.0,0.001210,44 days
50110421,67098556,1641864,570535104,2018-06-30,2018-08-13,0,0,0,37653.0,40829000.0,0.045926,44 days


2. Funds must have exactly one filing point for each quarter i.e., remove funds with amendments.

In [9]:
# counterexample
form13f_46[(form13f_46["iCIK"] == 1536562) & (form13f_46["iPERIOD_END"] == datetime.date(2014,12,31))]

,iRECORD_ID,iCIK,iCUSIP,iPERIOD_END,iFILING_DATE,iAMEND,iRESTATEMENT,iTYPE,iQTY,iMARKET_VALUE,iLONG_FRACTION,FILING_INTERVAL
26100887,50955873,1536562,438516106,2014-12-31,2015-02-02,0,0,0,9000.0,769000.0,0.020278,33 days
26100888,50955874,1536562,71654V101,2014-12-31,2015-02-02,0,0,0,98132.0,10440000.0,0.275295,33 days
26100889,50955875,1536562,05961W105,2014-12-31,2015-02-02,0,0,0,68521.0,3180000.0,0.083854,33 days
26100890,50955876,1536562,594918104,2014-12-31,2015-02-02,0,0,0,23000.0,1141000.0,0.030087,33 days
26100891,50955877,1536562,437076102,2014-12-31,2015-02-02,0,0,0,6200.0,626000.0,0.016507,33 days
26100892,50955878,1536562,81369Y407,2014-12-31,2015-02-02,0,0,0,16500.0,1163000.0,0.030667,33 days
26100893,50955879,1536562,03524A108,2014-12-31,2015-02-02,0,0,0,105428.0,11476000.0,0.302613,33 days
26100894,50955880,1536562,38259P706,2014-12-31,2015-02-02,0,0,0,3500.0,744000.0,0.019619,33 days
26100895,50955881,1536562,146229109,2014-12-31,2015-02-02,0,0,0,92790.0,8384000.0,0.221080,33 days
26119280,50974218,1536562,438516106,2014-12-31,2015-02-03,1,1,0,9000.0,899000.0,0.000000,34 days


In [10]:
# remove the whole fund for that filing period if amend/restate/type=1
form13f_amend = form13f_46.groupby(["iCIK", "iPERIOD_END"]).filter(lambda x:((x.iAMEND == 0)&(x.iRESTATEMENT == 0)&(x.iTYPE == 0)).all())
form13f_amend = form13f_amend.drop('iAMEND', axis=1)
form13f_amend = form13f_amend.drop('iRESTATEMENT', axis=1)
form13f_amend = form13f_amend.drop('iTYPE', axis=1)
form13f_amend

,iRECORD_ID,iCIK,iCUSIP,iPERIOD_END,iFILING_DATE,iQTY,iMARKET_VALUE,iLONG_FRACTION,FILING_INTERVAL
26088669,38587073,1349353,002824100,2013-06-30,2013-08-01,96510.0,3366000.0,0.023443,32 days
26088670,38587074,1349353,025816109,2013-06-30,2013-08-01,70214.0,5249000.0,0.036557,32 days
26088671,38587075,1349353,026874784,2013-06-30,2013-08-01,346115.0,15471000.0,0.107748,32 days
26088672,38587076,1349353,060505104,2013-06-30,2013-08-01,882340.0,11347000.0,0.079026,32 days
26088673,38587077,1349353,064058100,2013-06-30,2013-08-01,129065.0,3620000.0,0.025212,32 days
...,...,...,...,...,...,...,...,...,...
50110382,67098517,1641042,731094108,2018-06-30,2018-08-13,29000.0,683000.0,0.003574,44 days
50110383,67098518,1641042,85570W100,2018-06-30,2018-08-13,150000.0,5445000.0,0.028494,44 days
50110384,67098519,1641042,056752108,2018-06-30,2018-08-13,48222.0,11720000.0,0.061331,44 days
50110385,67098520,1641042,427746102,2018-06-30,2018-08-13,334331.0,12989000.0,0.067971,44 days


3. Funds must have between 20 and 200 holdings.

In [11]:
form13f_20_200 = form13f_amend.groupby(["iCIK", "iPERIOD_END"]).filter(lambda x: ((20 <= len(x))&(len(x) <= 200)))
form13f_20_200

,iRECORD_ID,iCIK,iCUSIP,iPERIOD_END,iFILING_DATE,iQTY,iMARKET_VALUE,iLONG_FRACTION,FILING_INTERVAL
26088669,38587073,1349353,002824100,2013-06-30,2013-08-01,96510.0,3366000.0,0.023443,32 days
26088670,38587074,1349353,025816109,2013-06-30,2013-08-01,70214.0,5249000.0,0.036557,32 days
26088671,38587075,1349353,026874784,2013-06-30,2013-08-01,346115.0,15471000.0,0.107748,32 days
26088672,38587076,1349353,060505104,2013-06-30,2013-08-01,882340.0,11347000.0,0.079026,32 days
26088673,38587077,1349353,064058100,2013-06-30,2013-08-01,129065.0,3620000.0,0.025212,32 days
...,...,...,...,...,...,...,...,...,...
50110382,67098517,1641042,731094108,2018-06-30,2018-08-13,29000.0,683000.0,0.003574,44 days
50110383,67098518,1641042,85570W100,2018-06-30,2018-08-13,150000.0,5445000.0,0.028494,44 days
50110384,67098519,1641042,056752108,2018-06-30,2018-08-13,48222.0,11720000.0,0.061331,44 days
50110385,67098520,1641042,427746102,2018-06-30,2018-08-13,334331.0,12989000.0,0.067971,44 days


4. Funds must be valued between 100M and 500M.

In [12]:
form13f_100_500 = form13f_20_200.groupby(["iCIK", "iPERIOD_END"]).filter(lambda x:((100000000 <= sum(x.iMARKET_VALUE))&(sum(x.iMARKET_VALUE) <= 500000000)))
form13f_100_500

,iRECORD_ID,iCIK,iCUSIP,iPERIOD_END,iFILING_DATE,iQTY,iMARKET_VALUE,iLONG_FRACTION,FILING_INTERVAL
26088669,38587073,1349353,002824100,2013-06-30,2013-08-01,96510.0,3366000.0,0.023443,32 days
26088670,38587074,1349353,025816109,2013-06-30,2013-08-01,70214.0,5249000.0,0.036557,32 days
26088671,38587075,1349353,026874784,2013-06-30,2013-08-01,346115.0,15471000.0,0.107748,32 days
26088672,38587076,1349353,060505104,2013-06-30,2013-08-01,882340.0,11347000.0,0.079026,32 days
26088673,38587077,1349353,064058100,2013-06-30,2013-08-01,129065.0,3620000.0,0.025212,32 days
...,...,...,...,...,...,...,...,...,...
50110382,67098517,1641042,731094108,2018-06-30,2018-08-13,29000.0,683000.0,0.003574,44 days
50110383,67098518,1641042,85570W100,2018-06-30,2018-08-13,150000.0,5445000.0,0.028494,44 days
50110384,67098519,1641042,056752108,2018-06-30,2018-08-13,48222.0,11720000.0,0.061331,44 days
50110385,67098520,1641042,427746102,2018-06-30,2018-08-13,334331.0,12989000.0,0.067971,44 days


5. Funds had to meet all filtering criteria for at least one year -- Survival funds

In [13]:
form13f_1year = form13f_100_500.groupby("iCIK").filter(lambda x:(x.iPERIOD_END.nunique() >= 4))
form13f_1year

,iRECORD_ID,iCIK,iCUSIP,iPERIOD_END,iFILING_DATE,iQTY,iMARKET_VALUE,iLONG_FRACTION,FILING_INTERVAL
26088669,38587073,1349353,002824100,2013-06-30,2013-08-01,96510.0,3366000.0,0.023443,32 days
26088670,38587074,1349353,025816109,2013-06-30,2013-08-01,70214.0,5249000.0,0.036557,32 days
26088671,38587075,1349353,026874784,2013-06-30,2013-08-01,346115.0,15471000.0,0.107748,32 days
26088672,38587076,1349353,060505104,2013-06-30,2013-08-01,882340.0,11347000.0,0.079026,32 days
26088673,38587077,1349353,064058100,2013-06-30,2013-08-01,129065.0,3620000.0,0.025212,32 days
...,...,...,...,...,...,...,...,...,...
50110382,67098517,1641042,731094108,2018-06-30,2018-08-13,29000.0,683000.0,0.003574,44 days
50110383,67098518,1641042,85570W100,2018-06-30,2018-08-13,150000.0,5445000.0,0.028494,44 days
50110384,67098519,1641042,056752108,2018-06-30,2018-08-13,48222.0,11720000.0,0.061331,44 days
50110385,67098520,1641042,427746102,2018-06-30,2018-08-13,334331.0,12989000.0,0.067971,44 days


In [14]:
print("number of survival funds:", len(form13f_1year["iCIK"].unique()))

number of survival funds: 1808


In [15]:
fund_stocks = form13f_1year["iCUSIP"].unique()
print(len(fund_stocks))
fund_stocks

17407


array(['002824100', '025816109', '026874784', ..., '86877M209',
       '30041R108', '83410S108'], dtype=object)

# II. FUND CLASSIFICATION AND FEATURE EXTRACTION

### 1. Industry Classification (correlation to sector ETFs)

In [16]:
prices = pd.read_csv("../RebellionData/rebellion_databases/prices.csv", sep=";")

In [17]:
prices["pSP_DATE"]= pd.to_datetime(prices["pSP_DATE"]).dt.date
prices.sort_values(by=["pSP_DATE"], inplace = True, ascending = True)

In [18]:
prices = prices[prices.pSP_DATE <= (PERIOD_TRAIN + datetime.timedelta(days=45))]

In [19]:
prices

,pSP_CUSIP,pSP_EXCHANGE,pSP_TICKER,pSP_DATE,pSP_VOLUME,pSP_OPEN,pSP_HIGH,pSP_LOW,pSP_CLOSE
9581087,73936Q405,N,PAO,2013-01-01,0,0.215,0.215,0.215,0.2150
9877577,73937B100,N,PKN,2013-01-01,0,46.700,46.700,46.700,46.6950
9882024,78464A805,N,TMW,2013-01-01,0,4.324,4.324,4.324,4.3240
9882494,73935X203,N,PWO,2013-01-01,0,24.640,24.640,24.640,24.6445
9583019,30304T106,N,FSG,2013-01-01,0,1.110,1.110,1.110,1.1120
...,...,...,...,...,...,...,...,...,...
25846775,055660104,PK,BPXXY,2018-08-14,0,10.350,10.350,10.350,10.3500
25818961,29272W109,N,ENR,2018-08-14,346337,63.370,64.080,63.370,63.5100
27414863,09057N300,OS,BPTH,2018-08-14,1335,29.400,30.598,28.000,28.2000
25832593,671044105,O,OSIS,2018-08-14,150875,78.800,79.755,78.480,79.3600


In [20]:
print("total number of stocks:", len(prices["pSP_CUSIP"].unique()))

total number of stocks: 29024


In [21]:
# prices of fund stocks
fund_prices = prices[prices["pSP_CUSIP"].isin(fund_stocks)]
len(fund_prices["pSP_CUSIP"].unique())

10073

In [22]:
XLF = prices.loc[prices["pSP_TICKER"] == "XLF" ]
XLY = prices.loc[prices["pSP_TICKER"] == "XLY" ]
XLK = prices.loc[prices["pSP_TICKER"] == "XLK" ]
XLB = prices.loc[prices["pSP_TICKER"] == "XLB" ]
XLI = prices.loc[prices["pSP_TICKER"] == "XLI" ]
XLE = prices.loc[prices["pSP_TICKER"] == "XLE" ]
XLU = prices.loc[prices["pSP_TICKER"] == "XLU" ]
XLV = prices.loc[prices["pSP_TICKER"] == "XLV" ]
XLP = prices.loc[prices["pSP_TICKER"] == "XLP" ]
VOX = prices.loc[prices["pSP_TICKER"] == "VOX" ]

In [23]:
print(len(XLF))
print(len(XLY))
print(len(XLK))
print(len(XLB))
print(len(XLI))
print(len(XLE))
print(len(XLU))
print(len(XLV))
print(len(XLP))
print(len(VOX))

1415
1415
1415
1415
1415
1415
1415
1415
1415
1415


In [24]:
XLF = XLF[XLF["pSP_DATE"].isin(VOX["pSP_DATE"])]
XLY = XLY[XLY["pSP_DATE"].isin(VOX["pSP_DATE"])]
XLK = XLK[XLK["pSP_DATE"].isin(VOX["pSP_DATE"])]
XLB = XLB[XLB["pSP_DATE"].isin(VOX["pSP_DATE"])]
XLI = XLI[XLI["pSP_DATE"].isin(VOX["pSP_DATE"])]
XLE = XLE[XLE["pSP_DATE"].isin(VOX["pSP_DATE"])]
XLU = XLU[XLU["pSP_DATE"].isin(VOX["pSP_DATE"])]
XLV = XLV[XLV["pSP_DATE"].isin(VOX["pSP_DATE"])]
XLP = XLP[XLP["pSP_DATE"].isin(VOX["pSP_DATE"])]

In [25]:
Etf_data = {'pSP_DATE':VOX["pSP_DATE"].values,
            'XLF':XLF["pSP_CLOSE"].values,
            'XLY':XLY["pSP_CLOSE"].values,
            'XLK':XLK["pSP_CLOSE"].values,
            'XLB':XLB["pSP_CLOSE"].values,
            'XLI':XLI["pSP_CLOSE"].values,
            'XLE':XLE["pSP_CLOSE"].values,
            'XLU':XLU["pSP_CLOSE"].values,
            'XLV':XLV["pSP_CLOSE"].values,
            'XLP':XLP["pSP_CLOSE"].values,
            'VOX':VOX["pSP_CLOSE"].values}

Etf_df = pd.DataFrame(Etf_data)
Etf_df

,pSP_DATE,XLF,XLY,XLK,XLB,XLI,XLE,XLU,XLV,XLP,VOX
0,2013-01-02,12.5558,45.5328,27.5843,35.2652,35.6120,66.0461,30.5576,38.1085,32.1762,63.7305
1,2013-01-03,12.5409,45.6456,27.4085,35.0086,35.6303,66.0280,30.5576,38.1554,32.0953,63.7931
2,2013-01-04,12.6897,45.8148,27.2974,35.4393,35.8964,66.7608,30.7123,38.3240,32.1852,64.2132
3,2013-01-07,12.6600,45.7020,27.2604,35.3660,35.7404,66.2451,30.4115,38.4458,31.9695,64.3651
4,2013-01-08,12.6228,45.5704,27.1586,35.4210,35.4652,66.0280,30.3170,38.4552,31.8796,63.3372
...,...,...,...,...,...,...,...,...,...,...,...
1410,2018-08-08,28.0255,112.0890,72.7892,59.2200,74.8967,73.7407,51.6824,89.0329,52.4541,85.6817
1411,2018-08-09,27.8582,112.5050,72.8189,59.5400,74.4837,73.0573,51.9165,88.9341,52.2975,86.4059
1412,2018-08-10,27.5334,111.5650,72.2353,58.6800,73.8248,73.5064,51.6922,88.5488,52.1018,85.8206
1413,2018-08-13,27.2676,111.0010,72.1364,58.0900,73.4708,72.5691,51.7605,88.5784,52.1018,85.6519


In [26]:
# ex. pSP_CUSIP == 032511107
x = fund_prices[fund_prices["pSP_CUSIP"] == "032511107" ]
joined = pd.merge(x, Etf_df, on = "pSP_DATE")
joined

,pSP_CUSIP,pSP_EXCHANGE,pSP_TICKER,pSP_DATE,pSP_VOLUME,pSP_OPEN,pSP_HIGH,pSP_LOW,pSP_CLOSE,XLF,XLY,XLK,XLB,XLI,XLE,XLU,XLV,XLP,VOX
0,032511107,N,APC,2013-01-02,4314603,75.75,76.080,74.7300,76.06,12.5558,45.5328,27.5843,35.2652,35.6120,66.0461,30.5576,38.1085,32.1762,63.7305
1,032511107,N,APC,2013-01-03,3098703,75.90,77.350,75.1600,76.33,12.5409,45.6456,27.4085,35.0086,35.6303,66.0280,30.5576,38.1554,32.0953,63.7931
2,032511107,N,APC,2013-01-04,3842372,76.59,78.480,76.4618,78.27,12.6897,45.8148,27.2974,35.4393,35.8964,66.7608,30.7123,38.3240,32.1852,64.2132
3,032511107,N,APC,2013-01-07,2744404,77.73,78.930,77.3500,78.25,12.6600,45.7020,27.2604,35.3660,35.7404,66.2451,30.4115,38.4458,31.9695,64.3651
4,032511107,N,APC,2013-01-08,3230932,77.83,78.860,77.2500,78.35,12.6228,45.5704,27.1586,35.4210,35.4652,66.0280,30.3170,38.4552,31.8796,63.3372
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1410,032511107,N,APC,2018-08-08,8035300,64.15,65.490,63.9300,65.15,28.0255,112.0890,72.7892,59.2200,74.8967,73.7407,51.6824,89.0329,52.4541,85.6817
1411,032511107,N,APC,2018-08-09,6255602,65.40,66.480,65.0900,66.06,27.8582,112.5050,72.8189,59.5400,74.4837,73.0573,51.9165,88.9341,52.2975,86.4059
1412,032511107,N,APC,2018-08-10,6780722,65.79,67.225,65.7400,66.82,27.5334,111.5650,72.2353,58.6800,73.8248,73.5064,51.6922,88.5488,52.1018,85.8206
1413,032511107,N,APC,2018-08-13,3853999,66.66,67.345,66.1900,66.34,27.2676,111.0010,72.1364,58.0900,73.4708,72.5691,51.7605,88.5784,52.1018,85.6519


In [27]:
# ex cont.
joined[joined.columns[8:]].corr()

,pSP_CLOSE,XLF,XLY,XLK,XLB,XLI,XLE,XLU,XLV,XLP,VOX
pSP_CLOSE,1.000000,-0.511283,-0.595998,-0.565372,-0.390128,-0.533597,0.892259,-0.712940,-0.601508,-0.737222,-0.519862
XLF,-0.511283,1.000000,0.959828,0.976953,0.958412,0.985776,-0.255251,0.834339,0.900246,0.791328,0.652038
XLY,-0.595998,0.959828,1.000000,0.980265,0.910810,0.966076,-0.356133,0.870533,0.946787,0.854977,0.688922
XLK,-0.565372,0.976953,0.980265,1.000000,0.941433,0.976983,-0.315408,0.859714,0.908857,0.801796,0.633513
XLB,-0.390128,0.958412,0.910810,0.941433,1.000000,0.973937,-0.085366,0.812116,0.880655,0.752658,0.654521
XLI,-0.533597,0.985776,0.966076,0.976983,0.973937,1.000000,-0.240482,0.880302,0.922608,0.839170,0.720131
XLE,0.892259,-0.255251,-0.356133,-0.315408,-0.085366,-0.240482,1.000000,-0.449875,-0.378554,-0.496175,-0.216151
XLU,-0.712940,0.834339,0.870533,0.859714,0.812116,0.880302,-0.449875,1.000000,0.896648,0.957261,0.856279
XLV,-0.601508,0.900246,0.946787,0.908857,0.880655,0.922608,-0.378554,0.896648,1.000000,0.911357,0.738329
XLP,-0.737222,0.791328,0.854977,0.801796,0.752658,0.839170,-0.496175,0.957261,0.911357,1.000000,0.892175


In [28]:
# ex cont.
joined[joined.columns[8:]].corr()["pSP_CLOSE"][1:]

XLF   -0.511283
XLY   -0.595998
XLK   -0.565372
XLB   -0.390128
XLI   -0.533597
XLE    0.892259
XLU   -0.712940
XLV   -0.601508
XLP   -0.737222
VOX   -0.519862
Name: pSP_CLOSE, dtype: float64

In [29]:
# group fund_prices data by pSP_CUSIP
# for every group, do merge and correlation
# calculate each stock's industry through a correlation script
stock_group = fund_prices.groupby("pSP_CUSIP")
sector_categories = pd.DataFrame(columns = ["iCUSIP", "sector"])
Headings = ["Financial", "Consumer Discretionary", "Technology", "Materials", "Industrial",
            "Energy", "Utilities", "Health Care", "Consumer Staples", "Communications"]

for i in fund_stocks:
    try:
        stock = stock_group.get_group(str(i))
        joined = pd.merge(stock, Etf_df, on = "pSP_DATE")
        sector = joined[joined.columns[8:]].corr()["pSP_CLOSE"][1:]
        sector_index = np.array(sector).argmax()
        sector_categories = sector_categories.append({'iCUSIP': i, 'sector':Headings[sector_index]}, ignore_index = True)
    except KeyError:
        sector_categories = sector_categories.append({'iCUSIP': i, 'sector':'NA'}, ignore_index = True)

In [30]:
sector_categories

,iCUSIP,sector
0,002824100,Technology
1,025816109,Materials
2,026874784,Consumer Staples
3,060505104,Financial
4,064058100,Financial
...,...,...
17402,05360L205,Communications
17403,05280R100,Utilities
17404,86877M209,Communications
17405,30041R108,Communications


In [31]:
form13f_sector = pd.merge(form13f_1year, sector_categories, on = "iCUSIP")
form13f_sector.sort_values(by=["iPERIOD_END", "iCIK", "sector"], inplace = True, ascending = True)
form13f_sector

,iRECORD_ID,iCIK,iCUSIP,iPERIOD_END,iFILING_DATE,iQTY,iMARKET_VALUE,iLONG_FRACTION,FILING_INTERVAL,sector
110010,39276322,7195,742718109,2013-06-30,2013-08-12,61112.0,4705000.0,0.020468,43 days,Communications
164917,39276303,7195,369604103,2013-06-30,2013-08-12,182959.0,4243000.0,0.018458,43 days,Communications
302628,39276331,7195,92343V104,2013-06-30,2013-08-12,94160.0,4740000.0,0.020620,43 days,Communications
213741,39276306,7195,437076102,2013-06-30,2013-08-12,64059.0,4963000.0,0.021590,43 days,Consumer Discretionary
286575,39276297,7195,22160K105,2013-06-30,2013-08-12,41274.0,4564000.0,0.019855,43 days,Consumer Discretionary
...,...,...,...,...,...,...,...,...,...,...
1655233,65812260,1721608,92189F445,2018-06-30,2018-07-13,49724.0,1212000.0,0.004417,13 days,NA
1657269,65812259,1721608,46434G772,2018-06-30,2018-07-13,260955.0,9394000.0,0.034237,13 days,NA
1255782,65812254,1721608,464288174,2018-06-30,2018-07-13,6167.0,482000.0,0.001757,13 days,Technology
1676712,65812273,1721608,46090E103,2018-06-30,2018-07-13,14237.0,2444000.0,0.008907,13 days,Technology


### 2. Value & Size Classification (correlation to Value and Size ETFs)

In [32]:
SPY = prices.loc[prices["pSP_TICKER"] == "SPY" ] # 1865
QQQ = prices[(prices["pSP_TICKER"] == "QQQ") & (prices["pSP_CUSIP"] == "46090E103") ] # 1865
IWD = prices.loc[prices["pSP_TICKER"] == "IWD" ] # 1865
IJH = prices.loc[prices["pSP_TICKER"] == "IJH" ] # 1856
IWP = prices.loc[prices["pSP_TICKER"] == "IWP" ] # 1857
IWS = prices.loc[prices["pSP_TICKER"] == "IWS" ] # 1865
IWM = prices.loc[prices["pSP_TICKER"] == "IWM" ] # 1865
IWO = prices.loc[prices["pSP_TICKER"] == "IWO" ] # 1865
VBR = prices.loc[prices["pSP_TICKER"] == "VBR" ] # 1865

In [33]:
print(len(SPY))
print(len(QQQ))
print(len(IWD))
print(len(IJH))
print(len(IWP))
print(len(IWS))
print(len(IWM))
print(len(IWO))
print(len(VBR))

1415
1415
1415
1415
1415
1415
1415
1415
1415


In [34]:
SPY = SPY[SPY["pSP_DATE"].isin(IJH["pSP_DATE"])]
QQQ = QQQ[QQQ["pSP_DATE"].isin(IJH["pSP_DATE"])]
IWD = IWD[IWD["pSP_DATE"].isin(IJH["pSP_DATE"])]
IWP = IWP[IWP["pSP_DATE"].isin(IJH["pSP_DATE"])]
IWS = IWS[IWS["pSP_DATE"].isin(IJH["pSP_DATE"])]
IWM = IWM[IWM["pSP_DATE"].isin(IJH["pSP_DATE"])]
IWO = IWO[IWO["pSP_DATE"].isin(IJH["pSP_DATE"])]
VBR = VBR[VBR["pSP_DATE"].isin(IJH["pSP_DATE"])]

In [35]:
Etf_data2 = {'pSP_DATE':SPY["pSP_DATE"].values,
             'SPY':SPY["pSP_CLOSE"].values,
             'QQQ':QQQ["pSP_CLOSE"].values,
             'IWD':IWD["pSP_CLOSE"].values,
             'IJH':IJH["pSP_CLOSE"].values,
             'IWP':IWP["pSP_CLOSE"].values,
             'IWS':IWS["pSP_CLOSE"].values,
             'IWM':IWM["pSP_CLOSE"].values,
             'IWO':IWO["pSP_CLOSE"].values,
             'VBR':VBR["pSP_CLOSE"].values}

Etf_df2 = pd.DataFrame(Etf_data2)
Etf_df2

,pSP_DATE,SPY,QQQ,IWD,IJH,IWP,IWS,IWM,IWO,VBR
0,2013-01-02,146.060,67.2000,67.8392,98.2093,61.8643,47.2322,81.8058,94.5759,69.0195
1,2013-01-03,145.734,66.8521,67.7392,98.2469,61.8547,47.2689,81.6173,94.5373,69.1488
2,2013-01-04,146.370,66.6300,68.2575,98.9322,62.2392,47.6732,82.2205,95.0677,69.6291
3,2013-01-07,145.969,66.6504,67.9574,98.6787,62.0662,47.4802,81.9094,94.9905,69.3058
4,2013-01-08,145.550,66.5200,67.7483,98.3971,61.9220,47.2322,81.8529,94.9713,69.1118
...,...,...,...,...,...,...,...,...,...,...
1410,2018-08-08,285.460,182.0200,123.8160,198.6080,130.9570,89.1317,166.1260,209.2600,137.9900
1411,2018-08-09,285.070,181.9100,123.5510,198.3800,131.1950,89.0629,166.5520,209.9860,137.8920
1412,2018-08-10,283.160,180.5200,122.6670,197.2620,130.5790,88.4434,166.0870,209.8770,137.1650
1413,2018-08-13,282.100,180.3200,121.9800,196.2330,129.9140,88.0108,165.1150,208.4540,136.4180


In [36]:
stock_group = fund_prices.groupby("pSP_CUSIP")
value_size_categories = pd.DataFrame(columns = ["iCUSIP", "value_size"])
Headings = ["Large-Cap Blend", "Large-Cap Growth", "Large-Cap Value", "MidCap Blend", "MidCap Growth",
            "MidCap Value", "Small-Cap Blend", "Small-Cap Growth", "Small-Cap Value"]

for i in fund_stocks:
    try:
        stock = stock_group.get_group(str(i))
        joined = pd.merge(stock, Etf_df2, on = "pSP_DATE")
        sector = joined[joined.columns[8:]].corr()["pSP_CLOSE"][1:]
        sector_index = np.array(sector).argmax()
        value_size_categories = value_size_categories.append({'iCUSIP': i, 'value_size':Headings[sector_index]}, ignore_index = True)
    except KeyError:
        value_size_categories = value_size_categories.append({'iCUSIP': i, 'value_size':'NA'}, ignore_index = True)

In [37]:
value_size_categories

,iCUSIP,value_size
0,002824100,MidCap Growth
1,025816109,Small-Cap Growth
2,026874784,Large-Cap Value
3,060505104,Small-Cap Blend
4,064058100,Large-Cap Blend
...,...,...
17402,05360L205,Large-Cap Growth
17403,05280R100,Small-Cap Value
17404,86877M209,Small-Cap Blend
17405,30041R108,Small-Cap Growth


In [38]:
form13f_value_size = pd.merge(form13f_sector, value_size_categories, on = "iCUSIP")
form13f_value_size.sort_values(by=["iPERIOD_END", "iCIK", "sector", "value_size"], inplace = True, ascending = True)
form13f_value_size

,iRECORD_ID,iCIK,iCUSIP,iPERIOD_END,iFILING_DATE,iQTY,iMARKET_VALUE,iLONG_FRACTION,FILING_INTERVAL,sector,value_size
0,39276322,7195,742718109,2013-06-30,2013-08-12,61112.0,4705000.0,0.020468,43 days,Communications,Large-Cap Value
21620,39276331,7195,92343V104,2013-06-30,2013-08-12,94160.0,4740000.0,0.020620,43 days,Communications,Large-Cap Value
10661,39276303,7195,369604103,2013-06-30,2013-08-12,182959.0,4243000.0,0.018458,43 days,Communications,MidCap Value
29832,39276306,7195,437076102,2013-06-30,2013-08-12,64059.0,4963000.0,0.021590,43 days,Consumer Discretionary,Large-Cap Growth
37004,39276297,7195,22160K105,2013-06-30,2013-08-12,41274.0,4564000.0,0.019855,43 days,Consumer Discretionary,Large-Cap Growth
...,...,...,...,...,...,...,...,...,...,...,...
1655232,65812260,1721608,92189F445,2018-06-30,2018-07-13,49724.0,1212000.0,0.004417,13 days,NA,NA
1657181,65812259,1721608,46434G772,2018-06-30,2018-07-13,260955.0,9394000.0,0.034237,13 days,NA,NA
1677147,65812273,1721608,46090E103,2018-06-30,2018-07-13,14237.0,2444000.0,0.008907,13 days,Technology,Large-Cap Growth
845372,65812254,1721608,464288174,2018-06-30,2018-07-13,6167.0,482000.0,0.001757,13 days,Technology,Small-Cap Growth


### 3. Performance (mean of funds' return as benchmark)

In [39]:
# closing price on filing date
form13f_close = pd.merge(form13f_value_size, fund_prices[["pSP_CLOSE","pSP_CUSIP","pSP_DATE"]],  how='left', left_on=["iCUSIP","iFILING_DATE"], right_on = ["pSP_CUSIP","pSP_DATE"])
form13f_close = form13f_close.iloc[:,:-2]
form13f_close

,iRECORD_ID,iCIK,iCUSIP,iPERIOD_END,iFILING_DATE,iQTY,iMARKET_VALUE,iLONG_FRACTION,FILING_INTERVAL,sector,value_size,pSP_CLOSE
0,39276322,7195,742718109,2013-06-30,2013-08-12,61112.0,4705000.0,0.020468,43 days,Communications,Large-Cap Value,81.6200
1,39276331,7195,92343V104,2013-06-30,2013-08-12,94160.0,4740000.0,0.020620,43 days,Communications,Large-Cap Value,49.6500
2,39276303,7195,369604103,2013-06-30,2013-08-12,182959.0,4243000.0,0.018458,43 days,Communications,MidCap Value,24.1915
3,39276306,7195,437076102,2013-06-30,2013-08-12,64059.0,4963000.0,0.021590,43 days,Consumer Discretionary,Large-Cap Growth,79.2000
4,39276297,7195,22160K105,2013-06-30,2013-08-12,41274.0,4564000.0,0.019855,43 days,Consumer Discretionary,Large-Cap Growth,115.7300
...,...,...,...,...,...,...,...,...,...,...,...,...
1678608,65812260,1721608,92189F445,2018-06-30,2018-07-13,49724.0,1212000.0,0.004417,13 days,NA,NA,NaN
1678609,65812259,1721608,46434G772,2018-06-30,2018-07-13,260955.0,9394000.0,0.034237,13 days,NA,NA,NaN
1678610,65812273,1721608,46090E103,2018-06-30,2018-07-13,14237.0,2444000.0,0.008907,13 days,Technology,Large-Cap Growth,179.6100
1678611,65812254,1721608,464288174,2018-06-30,2018-07-13,6167.0,482000.0,0.001757,13 days,Technology,Small-Cap Growth,76.9032


In [40]:
form13f_close["shares"] = form13f_close["iMARKET_VALUE"] / form13f_close["pSP_CLOSE"]
form13f_close

,iRECORD_ID,iCIK,iCUSIP,iPERIOD_END,iFILING_DATE,iQTY,iMARKET_VALUE,iLONG_FRACTION,FILING_INTERVAL,sector,value_size,pSP_CLOSE,shares
0,39276322,7195,742718109,2013-06-30,2013-08-12,61112.0,4705000.0,0.020468,43 days,Communications,Large-Cap Value,81.6200,57645.185004
1,39276331,7195,92343V104,2013-06-30,2013-08-12,94160.0,4740000.0,0.020620,43 days,Communications,Large-Cap Value,49.6500,95468.277946
2,39276303,7195,369604103,2013-06-30,2013-08-12,182959.0,4243000.0,0.018458,43 days,Communications,MidCap Value,24.1915,175392.183205
3,39276306,7195,437076102,2013-06-30,2013-08-12,64059.0,4963000.0,0.021590,43 days,Consumer Discretionary,Large-Cap Growth,79.2000,62664.141414
4,39276297,7195,22160K105,2013-06-30,2013-08-12,41274.0,4564000.0,0.019855,43 days,Consumer Discretionary,Large-Cap Growth,115.7300,39436.619718
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678608,65812260,1721608,92189F445,2018-06-30,2018-07-13,49724.0,1212000.0,0.004417,13 days,NA,NA,NaN,NaN
1678609,65812259,1721608,46434G772,2018-06-30,2018-07-13,260955.0,9394000.0,0.034237,13 days,NA,NA,NaN,NaN
1678610,65812273,1721608,46090E103,2018-06-30,2018-07-13,14237.0,2444000.0,0.008907,13 days,Technology,Large-Cap Growth,179.6100,13607.260175
1678611,65812254,1721608,464288174,2018-06-30,2018-07-13,6167.0,482000.0,0.001757,13 days,Technology,Small-Cap Growth,76.9032,6267.619553


In [41]:
form13f_close["FILING_INTERVAL"].astype('timedelta64[D]').mean() # mean filing interval

32.39370420698517

In [42]:
# assume filing interval is 32 days
# filing_3mos: previous filing date
# funds report完换股票
form13f_close["filing_3mos"] = form13f_close["iPERIOD_END"] - relativedelta(days=+60) # 31+30+31(previous period)-32(filing interval)
form13f_close

,iRECORD_ID,iCIK,iCUSIP,iPERIOD_END,iFILING_DATE,iQTY,iMARKET_VALUE,iLONG_FRACTION,FILING_INTERVAL,sector,value_size,pSP_CLOSE,shares,filing_3mos
0,39276322,7195,742718109,2013-06-30,2013-08-12,61112.0,4705000.0,0.020468,43 days,Communications,Large-Cap Value,81.6200,57645.185004,2013-05-01
1,39276331,7195,92343V104,2013-06-30,2013-08-12,94160.0,4740000.0,0.020620,43 days,Communications,Large-Cap Value,49.6500,95468.277946,2013-05-01
2,39276303,7195,369604103,2013-06-30,2013-08-12,182959.0,4243000.0,0.018458,43 days,Communications,MidCap Value,24.1915,175392.183205,2013-05-01
3,39276306,7195,437076102,2013-06-30,2013-08-12,64059.0,4963000.0,0.021590,43 days,Consumer Discretionary,Large-Cap Growth,79.2000,62664.141414,2013-05-01
4,39276297,7195,22160K105,2013-06-30,2013-08-12,41274.0,4564000.0,0.019855,43 days,Consumer Discretionary,Large-Cap Growth,115.7300,39436.619718,2013-05-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678608,65812260,1721608,92189F445,2018-06-30,2018-07-13,49724.0,1212000.0,0.004417,13 days,NA,NA,NaN,NaN,2018-05-01
1678609,65812259,1721608,46434G772,2018-06-30,2018-07-13,260955.0,9394000.0,0.034237,13 days,NA,NA,NaN,NaN,2018-05-01
1678610,65812273,1721608,46090E103,2018-06-30,2018-07-13,14237.0,2444000.0,0.008907,13 days,Technology,Large-Cap Growth,179.6100,13607.260175,2018-05-01
1678611,65812254,1721608,464288174,2018-06-30,2018-07-13,6167.0,482000.0,0.001757,13 days,Technology,Small-Cap Growth,76.9032,6267.619553,2018-05-01


In [43]:
# closing price 2 months before period end
form13f_3mos = pd.merge(form13f_close, fund_prices[["pSP_CLOSE","pSP_CUSIP","pSP_DATE"]],  how='left', left_on=["iCUSIP","filing_3mos"], right_on = ["pSP_CUSIP","pSP_DATE"])
form13f_3mos = form13f_3mos.iloc[:,:-2]
form13f_3mos

,iRECORD_ID,iCIK,iCUSIP,iPERIOD_END,iFILING_DATE,iQTY,iMARKET_VALUE,iLONG_FRACTION,FILING_INTERVAL,sector,value_size,pSP_CLOSE_x,shares,filing_3mos,pSP_CLOSE_y
0,39276322,7195,742718109,2013-06-30,2013-08-12,61112.0,4705000.0,0.020468,43 days,Communications,Large-Cap Value,81.6200,57645.185004,2013-05-01,76.9900
1,39276331,7195,92343V104,2013-06-30,2013-08-12,94160.0,4740000.0,0.020620,43 days,Communications,Large-Cap Value,49.6500,95468.277946,2013-05-01,52.4000
2,39276303,7195,369604103,2013-06-30,2013-08-12,182959.0,4243000.0,0.018458,43 days,Communications,MidCap Value,24.1915,175392.183205,2013-05-01,22.0784
3,39276306,7195,437076102,2013-06-30,2013-08-12,64059.0,4963000.0,0.021590,43 days,Consumer Discretionary,Large-Cap Growth,79.2000,62664.141414,2013-05-01,72.7600
4,39276297,7195,22160K105,2013-06-30,2013-08-12,41274.0,4564000.0,0.019855,43 days,Consumer Discretionary,Large-Cap Growth,115.7300,39436.619718,2013-05-01,108.2200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678608,65812260,1721608,92189F445,2018-06-30,2018-07-13,49724.0,1212000.0,0.004417,13 days,NA,NA,NaN,NaN,2018-05-01,NaN
1678609,65812259,1721608,46434G772,2018-06-30,2018-07-13,260955.0,9394000.0,0.034237,13 days,NA,NA,NaN,NaN,2018-05-01,NaN
1678610,65812273,1721608,46090E103,2018-06-30,2018-07-13,14237.0,2444000.0,0.008907,13 days,Technology,Large-Cap Growth,179.6100,13607.260175,2018-05-01,162.7800
1678611,65812254,1721608,464288174,2018-06-30,2018-07-13,6167.0,482000.0,0.001757,13 days,Technology,Small-Cap Growth,76.9032,6267.619553,2018-05-01,78.1677


In [44]:
# check nontrading date
p_date = fund_prices["pSP_DATE"].unique()
f_date = form13f_3mos[~form13f_3mos["filing_3mos"].isin(p_date)]["filing_3mos"].unique()
print(len(p_date))
print(len(f_date))
f_date

1449
5


array([datetime.date(2014, 11, 1), datetime.date(2015, 8, 1),
       datetime.date(2015, 11, 1), datetime.date(2016, 1, 31),
       datetime.date(2016, 5, 1)], dtype=object)

In [45]:
# check (so many) NaN
print(len(form13f_3mos[(form13f_3mos["pSP_CLOSE_y"].isna())]["iCIK"].unique()))
for i in form13f_3mos[(form13f_3mos["pSP_CLOSE_y"].isna())]["iCIK"].unique():
    f = form13f_3mos[(form13f_3mos["pSP_CLOSE_y"].isna()) & (form13f_3mos["iCIK"] == i) & (form13f_3mos["iPERIOD_END"] == datetime.date(2013,6,30))]
    print(i, len(f["iCUSIP"].unique()))

1778
18748 3
21175 5
65957 2
80835 1
84616 1
86312 1
101199 5
102212 2
312955 11
320376 23
712011 1
764112 1
774602 1
778963 28
779519 3
806097 12
809339 14
810672 20
810716 7
811360 1
811454 1
819535 1
846788 2
860189 1
860580 7
860645 10
860748 2
862853 20
866780 4
866995 1
868911 17
869367 9
872078 7
874791 5
878228 10
883790 12
884306 2
884310 12
884548 3
886920 18
894205 3
894300 1
898399 1
900169 8
902614 6
903064 18
911270 6
918504 14
919497 11
919530 2
920441 7
923469 1
924166 9
928204 5
928400 6
931328 1
932141 37
934745 17
937886 3
938506 2
938759 13
939791 5
939835 12
941560 5
943442 7
943712 5
945625 15
948985 15
1002152 14
1003279 5
1004744 1
1006435 1
1008877 4
1008895 34
1009950 1
1013234 4
1013701 5
1014306 2
1014315 3
1016150 1
1016683 17
1016972 6
1017115 5
1019231 2
1021117 32
1026303 1
1027799 2
1031972 7
1033974 2
1034012 1
1034188 27
1035912 12
1037558 7
1038661 29
1041241 1
1042063 6
1044924 2
1044929 17
1047339 15
1049648 3
1049650 40
1050417 3
1050463 22
105104

1598550 0
1598803 0
1599054 0
1599210 0
1599212 0
1599217 0
1599390 0
1599425 0
1599444 0
1599466 0
1599468 0
1599562 0
1599579 0
1599623 0
1599731 0
1599746 0
1599747 0
1599813 0
1599863 0
1600004 0
1600085 0
1600136 0
1600145 0
200724 0
357301 0
740918 0
752365 0
813470 0
910886 0
1068809 0
1084683 0
1091923 0
1092838 0
1106590 0
1119032 0
1133385 0
1166134 0
1169581 0
1260824 0
1280054 0
1308555 0
1315828 0
1351063 0
1353198 0
1353570 0
1387869 0
1389403 0
1407382 0
1453885 0
1456511 0
1483394 0
1485153 0
1500605 0
1506294 0
1510940 0
1512901 0
1513193 0
1525109 0
1527570 0
1531611 0
1533327 0
1536725 0
1543991 0
1555283 0
1555900 0
1569036 0
1569292 0
1570251 0
1576151 0
1580830 0
1581655 0
1582968 0
1584086 0
1584087 0
1584246 0
1585045 0
1585740 0
1593921 0
1595124 0
1595509 0
1595855 0
1597245 0
1598340 0
1598381 0
1599760 0
1599793 0
1599882 0
1600035 0
1600151 0
1600583 0
1600746 0
1600999 0
1601742 0
1601904 0
1602198 0
1602476 0
1603001 0
1603142 0
1603937 0
1606588 0
160723

1697717 0
1697790 0
1697796 0
1697847 0
1697855 0
1698051 0
811407 0
872359 0
899211 0
935359 0
944733 0
1041773 0
1109767 0
1122470 0
1218749 0
1279396 0
1353651 0
1367786 0
1438574 0
1473174 0
1532472 0
1535677 0
1536006 0
1536446 0
1572838 0
1576584 0
1578242 0
1580677 0
1591307 0
1606134 0
1616628 0
1626220 0
1632548 0
1638022 0
1656713 0
1657516 0
1661462 0
1665518 0
1665589 0
1666024 0
1669662 0
1671811 0
1672067 0
1672201 0
1683182 0
1683754 0
1689872 0
1690186 0
1691804 0
1691982 0
1694079 0
1694948 0
1695582 0
1695658 0
1696263 0
1696438 0
1696615 0
1696628 0
1696957 0
1697360 0
1697767 0
1697791 0
1697848 0
1697882 0
1698094 0
1698146 0
1698215 0
1698220 0
1699622 0
1699803 0
1703080 0
1704107 0
1705711 0
1706248 0
811808 0
820123 0
1054522 0
1343916 0
1425165 0
1634301 0
1652391 0
1656537 0
1675762 0
1680613 0
1681662 0
1688931 0
1694896 0
1697725 0
1698218 0
1703100 0
1706836 0
1709698 0
1710524 0
1713520 0
1713735 0
351262 0
1019531 0
1128239 0
1133999 0
1357993 0
1557689 

In [46]:
form13f_3mos["market_V_3mos"] = form13f_3mos["shares"] * form13f_3mos["pSP_CLOSE_y"]
form13f_3mos

,iRECORD_ID,iCIK,iCUSIP,iPERIOD_END,iFILING_DATE,iQTY,iMARKET_VALUE,iLONG_FRACTION,FILING_INTERVAL,sector,value_size,pSP_CLOSE_x,shares,filing_3mos,pSP_CLOSE_y,market_V_3mos
0,39276322,7195,742718109,2013-06-30,2013-08-12,61112.0,4705000.0,0.020468,43 days,Communications,Large-Cap Value,81.6200,57645.185004,2013-05-01,76.9900,4.438103e+06
1,39276331,7195,92343V104,2013-06-30,2013-08-12,94160.0,4740000.0,0.020620,43 days,Communications,Large-Cap Value,49.6500,95468.277946,2013-05-01,52.4000,5.002538e+06
2,39276303,7195,369604103,2013-06-30,2013-08-12,182959.0,4243000.0,0.018458,43 days,Communications,MidCap Value,24.1915,175392.183205,2013-05-01,22.0784,3.872379e+06
3,39276306,7195,437076102,2013-06-30,2013-08-12,64059.0,4963000.0,0.021590,43 days,Consumer Discretionary,Large-Cap Growth,79.2000,62664.141414,2013-05-01,72.7600,4.559443e+06
4,39276297,7195,22160K105,2013-06-30,2013-08-12,41274.0,4564000.0,0.019855,43 days,Consumer Discretionary,Large-Cap Growth,115.7300,39436.619718,2013-05-01,108.2200,4.267831e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678608,65812260,1721608,92189F445,2018-06-30,2018-07-13,49724.0,1212000.0,0.004417,13 days,NA,NA,NaN,NaN,2018-05-01,NaN,NaN
1678609,65812259,1721608,46434G772,2018-06-30,2018-07-13,260955.0,9394000.0,0.034237,13 days,NA,NA,NaN,NaN,2018-05-01,NaN,NaN
1678610,65812273,1721608,46090E103,2018-06-30,2018-07-13,14237.0,2444000.0,0.008907,13 days,Technology,Large-Cap Growth,179.6100,13607.260175,2018-05-01,162.7800,2.214990e+06
1678611,65812254,1721608,464288174,2018-06-30,2018-07-13,6167.0,482000.0,0.001757,13 days,Technology,Small-Cap Growth,76.9032,6267.619553,2018-05-01,78.1677,4.899254e+05


In [47]:
form13f_3mos_new = form13f_3mos.dropna()
form13f_3mos_new

,iRECORD_ID,iCIK,iCUSIP,iPERIOD_END,iFILING_DATE,iQTY,iMARKET_VALUE,iLONG_FRACTION,FILING_INTERVAL,sector,value_size,pSP_CLOSE_x,shares,filing_3mos,pSP_CLOSE_y,market_V_3mos
0,39276322,7195,742718109,2013-06-30,2013-08-12,61112.0,4705000.0,0.020468,43 days,Communications,Large-Cap Value,81.6200,57645.185004,2013-05-01,76.9900,4.438103e+06
1,39276331,7195,92343V104,2013-06-30,2013-08-12,94160.0,4740000.0,0.020620,43 days,Communications,Large-Cap Value,49.6500,95468.277946,2013-05-01,52.4000,5.002538e+06
2,39276303,7195,369604103,2013-06-30,2013-08-12,182959.0,4243000.0,0.018458,43 days,Communications,MidCap Value,24.1915,175392.183205,2013-05-01,22.0784,3.872379e+06
3,39276306,7195,437076102,2013-06-30,2013-08-12,64059.0,4963000.0,0.021590,43 days,Consumer Discretionary,Large-Cap Growth,79.2000,62664.141414,2013-05-01,72.7600,4.559443e+06
4,39276297,7195,22160K105,2013-06-30,2013-08-12,41274.0,4564000.0,0.019855,43 days,Consumer Discretionary,Large-Cap Growth,115.7300,39436.619718,2013-05-01,108.2200,4.267831e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678604,65812268,1721608,464286806,2018-06-30,2018-07-13,12898.0,387000.0,0.001410,13 days,Materials,Small-Cap Growth,30.6300,12634.671890,2018-05-01,31.5867,3.990876e+05
1678605,65812258,1721608,922042874,2018-06-30,2018-07-13,1178.0,66000.0,0.000241,13 days,Materials,Small-Cap Growth,56.1676,1175.054658,2018-05-01,56.9964,6.697389e+04
1678610,65812273,1721608,46090E103,2018-06-30,2018-07-13,14237.0,2444000.0,0.008907,13 days,Technology,Large-Cap Growth,179.6100,13607.260175,2018-05-01,162.7800,2.214990e+06
1678611,65812254,1721608,464288174,2018-06-30,2018-07-13,6167.0,482000.0,0.001757,13 days,Technology,Small-Cap Growth,76.9032,6267.619553,2018-05-01,78.1677,4.899254e+05


In [48]:
total_market_V = form13f_3mos_new.groupby(["iCIK","iPERIOD_END"],as_index=False).iMARKET_VALUE.sum()
total_market_V_3mos = form13f_3mos_new.groupby(["iCIK","iPERIOD_END"],as_index=False).market_V_3mos.sum()
Return = pd.merge(total_market_V,total_market_V_3mos, on=["iCIK","iPERIOD_END"])
Return["return"] = (Return["iMARKET_VALUE"] - Return["market_V_3mos"]) / Return["market_V_3mos"]
Return # return of each fund in each period

,iCIK,iPERIOD_END,iMARKET_VALUE,market_V_3mos,return
0,7195,2013-06-30,229871000.0,2.157608e+08,0.065397
1,7195,2013-09-30,220987000.0,2.191440e+08,0.008410
2,7195,2013-12-31,244650000.0,2.355210e+08,0.038761
3,7195,2014-03-31,243537000.0,2.359490e+08,0.032159
4,7195,2014-06-30,236104000.0,2.258708e+08,0.045306
...,...,...,...,...,...
15477,1719739,2018-06-30,145355000.0,1.339388e+08,0.085235
15478,1721608,2017-09-30,114173000.0,1.116571e+08,0.022532
15479,1721608,2017-12-31,160337000.0,1.587604e+08,0.009931
15480,1721608,2018-03-31,222791000.0,2.304703e+08,-0.033320


In [49]:
cik_mean_returns = Return.groupby("iCIK",as_index=False)["return"].mean()
cik_mean_returns # mean return of each fund

,iCIK,return
0,7195,0.025251
1,18748,0.010671
2,19475,0.050653
3,19481,0.060120
4,21175,0.020743
...,...,...
1801,1713735,0.030064
1802,1714341,0.012985
1803,1716399,0.021163
1804,1719739,0.039878


In [50]:
mean_return = cik_mean_returns["return"].mean()
mean_return # mean return of all funds

0.024716644832761658

In [51]:
def performance(row):
    if row['return'] > mean_return * 1.2:
        p = "high"
    elif row['return'] < mean_return * 0.8:
        p = "low"
    else:
        p = "medium"
    return p
cik_mean_returns["performance"] = cik_mean_returns.apply(performance, axis=1)
cik_mean_returns

,iCIK,return,performance
0,7195,0.025251,medium
1,18748,0.010671,low
2,19475,0.050653,high
3,19481,0.060120,high
4,21175,0.020743,medium
...,...,...,...
1801,1713735,0.030064,high
1802,1714341,0.012985,low
1803,1716399,0.021163,medium
1804,1719739,0.039878,high


In [52]:
len(form13f_3mos["iCIK"].unique())

1808

### 4. Volatility

In [53]:
# std of period returns for each fund (Return) -> take mean
volatility = Return.groupby("iCIK",as_index=False)["return"].std()
volatility.columns = ["iCIK", "volatility"]
volatility

,iCIK,volatility
0,7195,0.042513
1,18748,0.037731
2,19475,0.031249
3,19481,0.009252
4,21175,0.067871
...,...,...
1801,1713735,0.034755
1802,1714341,0.052941
1803,1716399,0.035538
1804,1719739,0.055073


In [54]:
mean_volatility = volatility["volatility"].mean()
mean_volatility

0.05209480122518896

In [55]:
def vol(row):
    if row['volatility'] > mean_volatility * 1.2:
        v = "high"
    elif row['volatility'] < mean_volatility * 0.8:
        v = "low"
    else:
        v = "medium"
    return v
volatility["volatility group"] = volatility.apply(vol, axis=1)
volatility

,iCIK,volatility,volatility group
0,7195,0.042513,medium
1,18748,0.037731,low
2,19475,0.031249,low
3,19481,0.009252,low
4,21175,0.067871,high
...,...,...,...
1801,1713735,0.034755,low
1802,1714341,0.052941,medium
1803,1716399,0.035538,low
1804,1719739,0.055073,medium


### 5. Survival Time

In [56]:
period_min = form13f_3mos.groupby("iCIK",as_index=False).iPERIOD_END.min()
period_min.columns = ["iCIK", "iPERIOD_END_min"]
period_max = form13f_3mos.groupby("iCIK",as_index=False).iPERIOD_END.max()
period_max.columns = ["iCIK", "iPERIOD_END_max"]
period = pd.merge(period_min, period_max, on=["iCIK"])
period["survival time"] = period["iPERIOD_END_max"] - period["iPERIOD_END_min"]
period

,iCIK,iPERIOD_END_min,iPERIOD_END_max,survival time
0,7195,2013-06-30,2018-06-30,1826 days
1,18748,2013-06-30,2016-09-30,1188 days
2,19475,2013-06-30,2018-06-30,1826 days
3,19481,2013-06-30,2015-03-31,639 days
4,21175,2013-06-30,2018-06-30,1826 days
...,...,...,...,...
1803,1713735,2017-06-30,2018-06-30,365 days
1804,1714341,2017-06-30,2018-06-30,365 days
1805,1716399,2017-09-30,2018-06-30,273 days
1806,1719739,2017-09-30,2018-06-30,273 days


In [57]:
mean_survival = period["survival time"].mean()
mean_survival

Timedelta('1155 days 18:27:04.778761056')

In [58]:
def survival(row):
    if row['survival time'] > mean_survival * 1.5: # no fund survives twice the mean survival time
        s = "high"
    elif row['survival time'] < mean_survival * 0.8:
        s = "low"
    else:
        s = "medium"
    return s
period["survival group"] = period.apply(survival, axis=1)
period

,iCIK,iPERIOD_END_min,iPERIOD_END_max,survival time,survival group
0,7195,2013-06-30,2018-06-30,1826 days,high
1,18748,2013-06-30,2016-09-30,1188 days,medium
2,19475,2013-06-30,2018-06-30,1826 days,high
3,19481,2013-06-30,2015-03-31,639 days,low
4,21175,2013-06-30,2018-06-30,1826 days,high
...,...,...,...,...,...
1803,1713735,2017-06-30,2018-06-30,365 days,low
1804,1714341,2017-06-30,2018-06-30,365 days,low
1805,1716399,2017-09-30,2018-06-30,273 days,low
1806,1719739,2017-09-30,2018-06-30,273 days,low


### 6. Idiosyncratic Risk

In [59]:
####### ?????????

### 7. Inflow & Turnover

In [60]:
form13f_3mos

,iRECORD_ID,iCIK,iCUSIP,iPERIOD_END,iFILING_DATE,iQTY,iMARKET_VALUE,iLONG_FRACTION,FILING_INTERVAL,sector,value_size,pSP_CLOSE_x,shares,filing_3mos,pSP_CLOSE_y,market_V_3mos
0,39276322,7195,742718109,2013-06-30,2013-08-12,61112.0,4705000.0,0.020468,43 days,Communications,Large-Cap Value,81.6200,57645.185004,2013-05-01,76.9900,4.438103e+06
1,39276331,7195,92343V104,2013-06-30,2013-08-12,94160.0,4740000.0,0.020620,43 days,Communications,Large-Cap Value,49.6500,95468.277946,2013-05-01,52.4000,5.002538e+06
2,39276303,7195,369604103,2013-06-30,2013-08-12,182959.0,4243000.0,0.018458,43 days,Communications,MidCap Value,24.1915,175392.183205,2013-05-01,22.0784,3.872379e+06
3,39276306,7195,437076102,2013-06-30,2013-08-12,64059.0,4963000.0,0.021590,43 days,Consumer Discretionary,Large-Cap Growth,79.2000,62664.141414,2013-05-01,72.7600,4.559443e+06
4,39276297,7195,22160K105,2013-06-30,2013-08-12,41274.0,4564000.0,0.019855,43 days,Consumer Discretionary,Large-Cap Growth,115.7300,39436.619718,2013-05-01,108.2200,4.267831e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678608,65812260,1721608,92189F445,2018-06-30,2018-07-13,49724.0,1212000.0,0.004417,13 days,NA,NA,NaN,NaN,2018-05-01,NaN,NaN
1678609,65812259,1721608,46434G772,2018-06-30,2018-07-13,260955.0,9394000.0,0.034237,13 days,NA,NA,NaN,NaN,2018-05-01,NaN,NaN
1678610,65812273,1721608,46090E103,2018-06-30,2018-07-13,14237.0,2444000.0,0.008907,13 days,Technology,Large-Cap Growth,179.6100,13607.260175,2018-05-01,162.7800,2.214990e+06
1678611,65812254,1721608,464288174,2018-06-30,2018-07-13,6167.0,482000.0,0.001757,13 days,Technology,Small-Cap Growth,76.9032,6267.619553,2018-05-01,78.1677,4.899254e+05


In [61]:
############################################ ex. iCIK == 7195 ##################################################
tp = form13f_3mos[form13f_3mos['iCIK'] == 7195]
tp = tp.sort_values(by = ["iCUSIP","iPERIOD_END"])
tp['buy'] = 0
tp

,iRECORD_ID,iCIK,iCUSIP,iPERIOD_END,iFILING_DATE,iQTY,iMARKET_VALUE,iLONG_FRACTION,FILING_INTERVAL,sector,value_size,pSP_CLOSE_x,shares,filing_3mos,pSP_CLOSE_y,market_V_3mos,buy
1589502,65860591,7195,00206R102,2018-06-30,2018-07-19,29025.0,932000.0,0.004254,19 days,Communications,MidCap Value,31.2700,29804.924848,2018-05-01,32.5400,9.698523e+05,0
1589566,65860611,7195,00287Y109,2018-06-30,2018-07-19,3656.0,339000.0,0.001547,19 days,Technology,MidCap Growth,89.9500,3768.760422,2018-05-01,102.0700,3.846774e+05,0
25,39276284,7195,009158106,2013-06-30,2013-08-12,46990.0,4303000.0,0.018719,43 days,Health Care,MidCap Value,95.9923,44826.512126,2013-05-01,79.7053,3.572911e+06,0
53604,40312064,7195,009158106,2013-09-30,2013-10-16,42995.0,4582000.0,0.020734,16 days,Health Care,MidCap Value,101.4710,45155.758788,2013-08-01,99.7595,4.504716e+06,0
113954,48527318,7195,009158106,2013-12-31,2014-01-21,42995.0,4806000.0,0.019558,21 days,Health Care,MidCap Value,104.2240,46112.219834,2013-11-01,100.4570,4.632295e+06,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32,39276308,7195,G47791101,2013-06-30,2013-08-12,85988.0,4774000.0,0.020768,43 days,Industrial,MidCap Blend,48.8932,97641.389805,2013-05-01,41.6114,4.062995e+06,0
53611,40312088,7195,G47791101,2013-09-30,2013-10-16,77953.0,5062000.0,0.022906,16 days,Industrial,MidCap Blend,50.4707,100295.815196,2013-08-01,49.4828,4.962918e+06,0
113961,48527304,7195,G47791101,2013-12-31,2014-01-21,77953.0,4802000.0,0.019541,21 days,Industrial,MidCap Blend,62.0700,77364.266151,2013-11-01,53.9682,4.175210e+06,0
186278,48825005,7195,G47791101,2014-03-31,2014-04-14,67158.0,3844000.0,0.015784,14 days,Industrial,MidCap Blend,55.2600,69562.070214,2014-01-30,59.0300,4.106249e+06,0


In [62]:
stocks = tp['iCUSIP'].unique()
for i in stocks:
    temp = tp[tp['iCUSIP'] == i]
    if len(temp) == 1:
        tp.at[temp.index[0], 'buy'] = temp.iloc[0]['iMARKET_VALUE']
    else:
        diff = np.diff(temp['iMARKET_VALUE'])
        diff = np.insert(diff, 0, temp.iloc[0]['iMARKET_VALUE'])
        for j in range(len(temp)):
            tp.at[temp.index[j],'buy'] = diff[j]
            
tp = tp.sort_values(by='iPERIOD_END')
tp

,iRECORD_ID,iCIK,iCUSIP,iPERIOD_END,iFILING_DATE,iQTY,iMARKET_VALUE,iLONG_FRACTION,FILING_INTERVAL,sector,value_size,pSP_CLOSE_x,shares,filing_3mos,pSP_CLOSE_y,market_V_3mos,buy
35,39276296,7195,219350105,2013-06-30,2013-08-12,302325.0,4302000.0,0.018715,43 days,Materials,Large-Cap Value,15.110,284712.111185,2013-05-01,14.330,4.079925e+06,4302000
29,39276307,7195,452308109,2013-06-30,2013-08-12,69101.0,4780000.0,0.020794,43 days,Industrial,Large-Cap Blend,73.520,65016.322089,2013-05-01,63.870,4.152592e+06,4780000
36,39276330,7195,907818108,2013-06-30,2013-08-12,33615.0,5186000.0,0.022561,43 days,Materials,MidCap Growth,79.585,65163.033235,2013-05-01,72.835,4.746150e+06,5186000
44,39276309,7195,458140100,2013-06-30,2013-08-12,189457.0,4591000.0,0.019972,43 days,Technology,Large-Cap Growth,22.640,202782.685512,2013-05-01,23.990,4.864757e+06,4591000
12,39276310,7195,459200101,2013-06-30,2013-08-12,21721.0,4151000.0,0.018058,43 days,Energy,Small-Cap Blend,189.090,21952.509387,2013-05-01,199.630,4.382379e+06,4151000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1589558,65860583,7195,67066G104,2018-06-30,2018-07-19,20400.0,4833000.0,0.022058,19 days,Technology,Large-Cap Growth,252.030,19176.288537,2018-05-01,227.140,4.355702e+06,4833000
1589520,65860604,7195,674599105,2018-06-30,2018-07-19,60265.0,5043000.0,0.023016,19 days,Energy,Small-Cap Growth,82.980,60773.680405,2018-05-01,76.580,4.654048e+06,991000
1589508,65860599,7195,713448108,2018-06-30,2018-07-19,39656.0,4317000.0,0.019703,19 days,Consumer Staples,Large-Cap Value,115.770,37289.453226,2018-05-01,99.130,3.696503e+06,292000
1589568,65860584,7195,615369105,2018-06-30,2018-07-19,1494.0,255000.0,0.001164,19 days,Technology,MidCap Growth,182.350,1398.409652,2018-05-01,164.160,2.295629e+05,255000


In [63]:
tp[(tp["iCUSIP"] == "882508104") & (tp["iPERIOD_END"] == datetime.date(2020,3,31))]

,iRECORD_ID,iCIK,iCUSIP,iPERIOD_END,iFILING_DATE,iQTY,iMARKET_VALUE,iLONG_FRACTION,FILING_INTERVAL,sector,value_size,pSP_CLOSE_x,shares,filing_3mos,pSP_CLOSE_y,market_V_3mos,buy


In [64]:
temp = tp['buy'].values
zeros = np.zeros_like(temp)
tp['sell'] = np.maximum(zeros, -temp)
tp['buy'] = np.maximum(zeros, temp)
tp

,iRECORD_ID,iCIK,iCUSIP,iPERIOD_END,iFILING_DATE,iQTY,iMARKET_VALUE,iLONG_FRACTION,FILING_INTERVAL,sector,value_size,pSP_CLOSE_x,shares,filing_3mos,pSP_CLOSE_y,market_V_3mos,buy,sell
35,39276296,7195,219350105,2013-06-30,2013-08-12,302325.0,4302000.0,0.018715,43 days,Materials,Large-Cap Value,15.110,284712.111185,2013-05-01,14.330,4.079925e+06,4302000,0
29,39276307,7195,452308109,2013-06-30,2013-08-12,69101.0,4780000.0,0.020794,43 days,Industrial,Large-Cap Blend,73.520,65016.322089,2013-05-01,63.870,4.152592e+06,4780000,0
36,39276330,7195,907818108,2013-06-30,2013-08-12,33615.0,5186000.0,0.022561,43 days,Materials,MidCap Growth,79.585,65163.033235,2013-05-01,72.835,4.746150e+06,5186000,0
44,39276309,7195,458140100,2013-06-30,2013-08-12,189457.0,4591000.0,0.019972,43 days,Technology,Large-Cap Growth,22.640,202782.685512,2013-05-01,23.990,4.864757e+06,4591000,0
12,39276310,7195,459200101,2013-06-30,2013-08-12,21721.0,4151000.0,0.018058,43 days,Energy,Small-Cap Blend,189.090,21952.509387,2013-05-01,199.630,4.382379e+06,4151000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1589558,65860583,7195,67066G104,2018-06-30,2018-07-19,20400.0,4833000.0,0.022058,19 days,Technology,Large-Cap Growth,252.030,19176.288537,2018-05-01,227.140,4.355702e+06,4833000,0
1589520,65860604,7195,674599105,2018-06-30,2018-07-19,60265.0,5043000.0,0.023016,19 days,Energy,Small-Cap Growth,82.980,60773.680405,2018-05-01,76.580,4.654048e+06,991000,0
1589508,65860599,7195,713448108,2018-06-30,2018-07-19,39656.0,4317000.0,0.019703,19 days,Consumer Staples,Large-Cap Value,115.770,37289.453226,2018-05-01,99.130,3.696503e+06,292000,0
1589568,65860584,7195,615369105,2018-06-30,2018-07-19,1494.0,255000.0,0.001164,19 days,Technology,MidCap Growth,182.350,1398.409652,2018-05-01,164.160,2.295629e+05,255000,0


In [65]:
grouped = tp.groupby(["iPERIOD_END"]).sum()
grouped['turnover'] = np.minimum(grouped['buy'], grouped['sell'])/grouped['iMARKET_VALUE']
grouped = grouped['turnover']
grouped = grouped.reset_index()
grouped['iCIK'] = 7195
grouped

,iPERIOD_END,turnover,iCIK
0,2013-06-30,0.000000,7195
1,2013-09-30,0.025201,7195
2,2013-12-31,0.001058,7195
3,2014-03-31,0.026546,7195
4,2014-06-30,0.050941,7195
5,2014-09-30,0.028756,7195
6,2014-12-31,0.015475,7195
7,2015-03-31,0.012458,7195
8,2015-06-30,0.020293,7195
9,2015-09-30,0.047148,7195


In [66]:
##################################################################################################################

In [67]:
# Quarterly turnover for each fund
def turnover_one_fund(df, index):
    tp = df[df['iCIK'] == index]
    tp = tp.sort_values(by = ["iCUSIP","iPERIOD_END"])
    tp['buy'] = 0
    stocks = tp['iCUSIP'].unique()
    for i in stocks:
        temp = tp[tp['iCUSIP'] == i]
        if len(temp) == 1:
            tp.at[temp.index[0], 'buy'] = temp.iloc[0]['iMARKET_VALUE']
        else:
            diff = np.diff(temp['iMARKET_VALUE'])
            diff = np.insert(diff, 0, temp.iloc[0]['iMARKET_VALUE'])
            for j in range(len(temp)):
                tp.at[temp.index[j],'buy'] = diff[j]
    tp = tp.sort_values(by='iPERIOD_END')
    temp = tp['buy'].values
    zeros = np.zeros_like(temp)
    tp['sell'] = np.maximum(zeros, -temp)
    tp['buy'] = np.maximum(zeros, temp)
    grouped = tp.groupby(["iPERIOD_END"]).sum()
    grouped['turnover'] = np.minimum(grouped['buy'], grouped['sell'])/grouped['iMARKET_VALUE']
    grouped = grouped['turnover']
    grouped = grouped.reset_index()
    grouped['iCIK'] = index
    return tp, grouped

In [68]:
form13f_inflow = []
Turnover = []

for i in form13f_3mos['iCIK'].unique():
    inflow, turnover = turnover_one_fund(form13f_3mos, i)
    form13f_inflow.append(inflow)
    Turnover.append(turnover)

form13f_inflow = pd.concat(form13f_inflow)
Turnover = pd.concat(Turnover)

In [69]:
form13f_inflow

,iRECORD_ID,iCIK,iCUSIP,iPERIOD_END,iFILING_DATE,iQTY,iMARKET_VALUE,iLONG_FRACTION,FILING_INTERVAL,sector,value_size,pSP_CLOSE_x,shares,filing_3mos,pSP_CLOSE_y,market_V_3mos,buy,sell
35,39276296,7195,219350105,2013-06-30,2013-08-12,302325.0,4302000.0,0.018715,43 days,Materials,Large-Cap Value,15.1100,284712.111185,2013-05-01,14.3300,4.079925e+06,4302000,0
29,39276307,7195,452308109,2013-06-30,2013-08-12,69101.0,4780000.0,0.020794,43 days,Industrial,Large-Cap Blend,73.5200,65016.322089,2013-05-01,63.8700,4.152592e+06,4780000,0
36,39276330,7195,907818108,2013-06-30,2013-08-12,33615.0,5186000.0,0.022561,43 days,Materials,MidCap Growth,79.5850,65163.033235,2013-05-01,72.8350,4.746150e+06,5186000,0
44,39276309,7195,458140100,2013-06-30,2013-08-12,189457.0,4591000.0,0.019972,43 days,Technology,Large-Cap Growth,22.6400,202782.685512,2013-05-01,23.9900,4.864757e+06,4591000,0
12,39276310,7195,459200101,2013-06-30,2013-08-12,21721.0,4151000.0,0.018058,43 days,Energy,Small-Cap Blend,189.0900,21952.509387,2013-05-01,199.6300,4.382379e+06,4151000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678596,65812266,1721608,78463X863,2018-06-30,2018-07-13,8483.0,332000.0,0.001210,13 days,Health Care,MidCap Value,37.8504,8771.373618,2018-05-01,38.7655,3.400267e+05,20000,0
1678584,65812256,1721608,78467V608,2018-06-30,2018-07-13,174973.0,8218000.0,0.029951,13 days,Communications,MidCap Value,45.2506,181610.851569,2018-05-01,45.0628,8.183893e+06,6341000,0
1678583,65812284,1721608,78468R408,2018-06-30,2018-07-13,31911.0,871000.0,0.003174,13 days,Communications,MidCap Value,26.2391,33194.736100,2018-05-01,25.9226,8.604939e+05,871000,0
1678606,65812253,1721608,92189F494,2018-06-30,2018-07-13,83255.0,1417000.0,0.005164,13 days,NA,NA,NaN,NaN,2018-05-01,NaN,NaN,0,98000


In [70]:
Turnover.sort_values(by='iPERIOD_END')
Turnover # turnover for each fund in each period

,iPERIOD_END,turnover,iCIK
0,2013-06-30,0.000000,7195
1,2013-09-30,0.025201,7195
2,2013-12-31,0.001058,7195
3,2014-03-31,0.026546,7195
4,2014-06-30,0.050941,7195
...,...,...,...
3,2018-06-30,0.018762,1719739
0,2017-09-30,0.000000,1721608
1,2017-12-31,0.000000,1721608
2,2018-03-31,0.066387,1721608


In [71]:
turnover_mean = Turnover.groupby('iPERIOD_END').mean()
turnover_mean

,turnover,iCIK
iPERIOD_END,,
2013-06-30,0.047669,1.271251e+06
2013-09-30,0.082798,1.286632e+06
2013-12-31,0.031695,1.316943e+06
2014-03-31,0.056569,1.323565e+06
2014-06-30,0.052065,1.335759e+06
2014-09-30,0.065565,1.332481e+06
2014-12-31,0.061723,1.333539e+06
2015-03-31,0.064917,1.347450e+06
2015-06-30,0.062884,1.351731e+06


In [72]:
def f(row):
    if row['turnover'] > turnover_mean.loc[row['iPERIOD_END']]['turnover'] * 1.5:
        t = "high"
    elif row['turnover'] < turnover_mean.loc[row['iPERIOD_END']]['turnover'] * 0.8:
        t = "low"
    else:
        t = "medium"
    return t
Turnover["turnover group"] = Turnover.apply(f, axis=1)
Turnover

,iPERIOD_END,turnover,iCIK,turnover group
0,2013-06-30,0.000000,7195,low
1,2013-09-30,0.025201,7195,low
2,2013-12-31,0.001058,7195,low
3,2014-03-31,0.026546,7195,low
4,2014-06-30,0.050941,7195,medium
...,...,...,...,...
3,2018-06-30,0.018762,1719739,low
0,2017-09-30,0.000000,1721608,low
1,2017-12-31,0.000000,1721608,low
2,2018-03-31,0.066387,1721608,medium


In [73]:
Turnover.sort_values(by='iPERIOD_END')
Turnover

,iPERIOD_END,turnover,iCIK,turnover group
0,2013-06-30,0.000000,7195,low
1,2013-09-30,0.025201,7195,low
2,2013-12-31,0.001058,7195,low
3,2014-03-31,0.026546,7195,low
4,2014-06-30,0.050941,7195,medium
...,...,...,...,...
3,2018-06-30,0.018762,1719739,low
0,2017-09-30,0.000000,1721608,low
1,2017-12-31,0.000000,1721608,low
2,2018-03-31,0.066387,1721608,medium


# Further Filtering

### 1. Turnover: funds with low / medium quarterly turnover

In [74]:
Turnover_filtered = Turnover.groupby("iCIK").filter(lambda x:((x["turnover group"] == "low") | (x["turnover group"] == "medium")).all())
Turnover_filtered

,iPERIOD_END,turnover,iCIK,turnover group
0,2013-06-30,0.000000,7195,low
1,2013-09-30,0.025201,7195,low
2,2013-12-31,0.001058,7195,low
3,2014-03-31,0.026546,7195,low
4,2014-06-30,0.050941,7195,medium
...,...,...,...,...
3,2018-06-30,0.018762,1719739,low
0,2017-09-30,0.000000,1721608,low
1,2017-12-31,0.000000,1721608,low
2,2018-03-31,0.066387,1721608,medium


In [75]:
form13f_FF1 = form13f_inflow[form13f_inflow.iCIK.isin(Turnover_filtered.iCIK.unique())]
form13f_FF1

,iRECORD_ID,iCIK,iCUSIP,iPERIOD_END,iFILING_DATE,iQTY,iMARKET_VALUE,iLONG_FRACTION,FILING_INTERVAL,sector,value_size,pSP_CLOSE_x,shares,filing_3mos,pSP_CLOSE_y,market_V_3mos,buy,sell
35,39276296,7195,219350105,2013-06-30,2013-08-12,302325.0,4302000.0,0.018715,43 days,Materials,Large-Cap Value,15.1100,284712.111185,2013-05-01,14.3300,4.079925e+06,4302000,0
29,39276307,7195,452308109,2013-06-30,2013-08-12,69101.0,4780000.0,0.020794,43 days,Industrial,Large-Cap Blend,73.5200,65016.322089,2013-05-01,63.8700,4.152592e+06,4780000,0
36,39276330,7195,907818108,2013-06-30,2013-08-12,33615.0,5186000.0,0.022561,43 days,Materials,MidCap Growth,79.5850,65163.033235,2013-05-01,72.8350,4.746150e+06,5186000,0
44,39276309,7195,458140100,2013-06-30,2013-08-12,189457.0,4591000.0,0.019972,43 days,Technology,Large-Cap Growth,22.6400,202782.685512,2013-05-01,23.9900,4.864757e+06,4591000,0
12,39276310,7195,459200101,2013-06-30,2013-08-12,21721.0,4151000.0,0.018058,43 days,Energy,Small-Cap Blend,189.0900,21952.509387,2013-05-01,199.6300,4.382379e+06,4151000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678596,65812266,1721608,78463X863,2018-06-30,2018-07-13,8483.0,332000.0,0.001210,13 days,Health Care,MidCap Value,37.8504,8771.373618,2018-05-01,38.7655,3.400267e+05,20000,0
1678584,65812256,1721608,78467V608,2018-06-30,2018-07-13,174973.0,8218000.0,0.029951,13 days,Communications,MidCap Value,45.2506,181610.851569,2018-05-01,45.0628,8.183893e+06,6341000,0
1678583,65812284,1721608,78468R408,2018-06-30,2018-07-13,31911.0,871000.0,0.003174,13 days,Communications,MidCap Value,26.2391,33194.736100,2018-05-01,25.9226,8.604939e+05,871000,0
1678606,65812253,1721608,92189F494,2018-06-30,2018-07-13,83255.0,1417000.0,0.005164,13 days,NA,NA,NaN,NaN,2018-05-01,NaN,NaN,0,98000


### 2. Survival time: funds with high / medium survival times

In [76]:
period_filtered = period[(period["survival group"] == "high") | (period["survival group"] == "medium")]
period_filtered

,iCIK,iPERIOD_END_min,iPERIOD_END_max,survival time,survival group
0,7195,2013-06-30,2018-06-30,1826 days,high
1,18748,2013-06-30,2016-09-30,1188 days,medium
2,19475,2013-06-30,2018-06-30,1826 days,high
4,21175,2013-06-30,2018-06-30,1826 days,high
5,36066,2013-12-31,2018-06-30,1642 days,medium
...,...,...,...,...,...
1591,1645890,2015-06-30,2018-06-30,1096 days,medium
1594,1648101,2015-06-30,2018-06-30,1096 days,medium
1599,1649888,2015-06-30,2018-06-30,1096 days,medium
1602,1650654,2015-06-30,2018-06-30,1096 days,medium


In [77]:
form13f_FF2 = form13f_FF1[form13f_FF1.iCIK.isin(period_filtered.iCIK.unique())]
form13f_FF2

,iRECORD_ID,iCIK,iCUSIP,iPERIOD_END,iFILING_DATE,iQTY,iMARKET_VALUE,iLONG_FRACTION,FILING_INTERVAL,sector,value_size,pSP_CLOSE_x,shares,filing_3mos,pSP_CLOSE_y,market_V_3mos,buy,sell
35,39276296,7195,219350105,2013-06-30,2013-08-12,302325.0,4302000.0,0.018715,43 days,Materials,Large-Cap Value,15.110,284712.111185,2013-05-01,14.330,4.079925e+06,4302000,0
29,39276307,7195,452308109,2013-06-30,2013-08-12,69101.0,4780000.0,0.020794,43 days,Industrial,Large-Cap Blend,73.520,65016.322089,2013-05-01,63.870,4.152592e+06,4780000,0
36,39276330,7195,907818108,2013-06-30,2013-08-12,33615.0,5186000.0,0.022561,43 days,Materials,MidCap Growth,79.585,65163.033235,2013-05-01,72.835,4.746150e+06,5186000,0
44,39276309,7195,458140100,2013-06-30,2013-08-12,189457.0,4591000.0,0.019972,43 days,Technology,Large-Cap Growth,22.640,202782.685512,2013-05-01,23.990,4.864757e+06,4591000,0
12,39276310,7195,459200101,2013-06-30,2013-08-12,21721.0,4151000.0,0.018058,43 days,Energy,Small-Cap Blend,189.090,21952.509387,2013-05-01,199.630,4.382379e+06,4151000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1664701,65828192,1653866,406216101,2018-06-30,2018-07-17,25045.0,1129000.0,0.002557,17 days,Energy,Small-Cap Blend,45.060,25055.481580,2018-05-01,52.440,1.313909e+06,0,67000
1664712,65828136,1653866,78467X109,2018-06-30,2018-07-17,6346.0,1540000.0,0.003488,17 days,Financial,Large-Cap Blend,251.280,6128.621458,2018-05-01,240.760,1.475527e+06,8000,0
1664711,65828101,1653866,06740L444,2018-06-30,2018-07-17,74198.0,1407000.0,0.003187,17 days,Energy,Small-Cap Growth,19.330,72788.411795,2018-05-01,20.840,1.516911e+06,0,1000
1664673,65828107,1653866,370334104,2018-06-30,2018-07-17,10823.0,479000.0,0.001085,17 days,Communications,MidCap Value,44.230,10829.753561,2018-05-01,43.550,4.716358e+05,479000,0


### 3. Volatility: funds with low / medium volatility

In [78]:
volatility_filtered = volatility[(volatility["volatility group"] == "low") | (volatility["volatility group"] == "medium")]
volatility_filtered

,iCIK,volatility,volatility group
0,7195,0.042513,medium
1,18748,0.037731,low
2,19475,0.031249,low
3,19481,0.009252,low
5,36066,0.033806,low
...,...,...,...
1801,1713735,0.034755,low
1802,1714341,0.052941,medium
1803,1716399,0.035538,low
1804,1719739,0.055073,medium


In [79]:
form13f_FF3 = form13f_FF2[form13f_FF2.iCIK.isin(volatility_filtered.iCIK.unique())]
form13f_FF3

,iRECORD_ID,iCIK,iCUSIP,iPERIOD_END,iFILING_DATE,iQTY,iMARKET_VALUE,iLONG_FRACTION,FILING_INTERVAL,sector,value_size,pSP_CLOSE_x,shares,filing_3mos,pSP_CLOSE_y,market_V_3mos,buy,sell
35,39276296,7195,219350105,2013-06-30,2013-08-12,302325.0,4302000.0,0.018715,43 days,Materials,Large-Cap Value,15.110,284712.111185,2013-05-01,14.330,4.079925e+06,4302000,0
29,39276307,7195,452308109,2013-06-30,2013-08-12,69101.0,4780000.0,0.020794,43 days,Industrial,Large-Cap Blend,73.520,65016.322089,2013-05-01,63.870,4.152592e+06,4780000,0
36,39276330,7195,907818108,2013-06-30,2013-08-12,33615.0,5186000.0,0.022561,43 days,Materials,MidCap Growth,79.585,65163.033235,2013-05-01,72.835,4.746150e+06,5186000,0
44,39276309,7195,458140100,2013-06-30,2013-08-12,189457.0,4591000.0,0.019972,43 days,Technology,Large-Cap Growth,22.640,202782.685512,2013-05-01,23.990,4.864757e+06,4591000,0
12,39276310,7195,459200101,2013-06-30,2013-08-12,21721.0,4151000.0,0.018058,43 days,Energy,Small-Cap Blend,189.090,21952.509387,2013-05-01,199.630,4.382379e+06,4151000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1664701,65828192,1653866,406216101,2018-06-30,2018-07-17,25045.0,1129000.0,0.002557,17 days,Energy,Small-Cap Blend,45.060,25055.481580,2018-05-01,52.440,1.313909e+06,0,67000
1664712,65828136,1653866,78467X109,2018-06-30,2018-07-17,6346.0,1540000.0,0.003488,17 days,Financial,Large-Cap Blend,251.280,6128.621458,2018-05-01,240.760,1.475527e+06,8000,0
1664711,65828101,1653866,06740L444,2018-06-30,2018-07-17,74198.0,1407000.0,0.003187,17 days,Energy,Small-Cap Growth,19.330,72788.411795,2018-05-01,20.840,1.516911e+06,0,1000
1664673,65828107,1653866,370334104,2018-06-30,2018-07-17,10823.0,479000.0,0.001085,17 days,Communications,MidCap Value,44.230,10829.753561,2018-05-01,43.550,4.716358e+05,479000,0


### 4. performance: funds with high / medium cik_mean_returns

In [80]:
cik_mean_returns_filtered = cik_mean_returns[(cik_mean_returns["performance"] == "high") | (cik_mean_returns["performance"] == "medium")]
cik_mean_returns_filtered

,iCIK,return,performance
0,7195,0.025251,medium
2,19475,0.050653,high
3,19481,0.060120,high
4,21175,0.020743,medium
5,36066,0.023635,medium
...,...,...,...
1797,1712671,0.020454,medium
1799,1713521,0.032111,high
1801,1713735,0.030064,high
1803,1716399,0.021163,medium


In [81]:
form13f_FF4 = form13f_FF3[form13f_FF3.iCIK.isin(cik_mean_returns_filtered.iCIK.unique())]
form13f_FF4

,iRECORD_ID,iCIK,iCUSIP,iPERIOD_END,iFILING_DATE,iQTY,iMARKET_VALUE,iLONG_FRACTION,FILING_INTERVAL,sector,value_size,pSP_CLOSE_x,shares,filing_3mos,pSP_CLOSE_y,market_V_3mos,buy,sell
35,39276296,7195,219350105,2013-06-30,2013-08-12,302325.0,4302000.0,0.018715,43 days,Materials,Large-Cap Value,15.110,284712.111185,2013-05-01,14.330,4.079925e+06,4302000,0
29,39276307,7195,452308109,2013-06-30,2013-08-12,69101.0,4780000.0,0.020794,43 days,Industrial,Large-Cap Blend,73.520,65016.322089,2013-05-01,63.870,4.152592e+06,4780000,0
36,39276330,7195,907818108,2013-06-30,2013-08-12,33615.0,5186000.0,0.022561,43 days,Materials,MidCap Growth,79.585,65163.033235,2013-05-01,72.835,4.746150e+06,5186000,0
44,39276309,7195,458140100,2013-06-30,2013-08-12,189457.0,4591000.0,0.019972,43 days,Technology,Large-Cap Growth,22.640,202782.685512,2013-05-01,23.990,4.864757e+06,4591000,0
12,39276310,7195,459200101,2013-06-30,2013-08-12,21721.0,4151000.0,0.018058,43 days,Energy,Small-Cap Blend,189.090,21952.509387,2013-05-01,199.630,4.382379e+06,4151000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1664701,65828192,1653866,406216101,2018-06-30,2018-07-17,25045.0,1129000.0,0.002557,17 days,Energy,Small-Cap Blend,45.060,25055.481580,2018-05-01,52.440,1.313909e+06,0,67000
1664712,65828136,1653866,78467X109,2018-06-30,2018-07-17,6346.0,1540000.0,0.003488,17 days,Financial,Large-Cap Blend,251.280,6128.621458,2018-05-01,240.760,1.475527e+06,8000,0
1664711,65828101,1653866,06740L444,2018-06-30,2018-07-17,74198.0,1407000.0,0.003187,17 days,Energy,Small-Cap Growth,19.330,72788.411795,2018-05-01,20.840,1.516911e+06,0,1000
1664673,65828107,1653866,370334104,2018-06-30,2018-07-17,10823.0,479000.0,0.001085,17 days,Communications,MidCap Value,44.230,10829.753561,2018-05-01,43.550,4.716358e+05,479000,0


In [82]:
print("number of funds after further filtering:", len(form13f_FF4.iCIK.unique()))

number of funds after further filtering: 227


In [83]:
# remaining stocks in fund_prices
fund_prices_FF = fund_prices[fund_prices.pSP_CUSIP.isin(form13f_FF4.iCUSIP.unique())]
len(fund_prices_FF.pSP_CUSIP.unique())

4792

# Files for feature engineering (EXHIBIT 7)

In [84]:
form13f_FF4.to_csv("../output/form13f_FF4.csv", sep='\t', index=False)

In [85]:
fund_prices_FF.to_csv("../output/fund_prices_FF.csv", sep='\t', index=False)

In [86]:
form13f[form13f.iPERIOD_END >= datetime.date(2013,6,30)].iPERIOD_END.unique()

array([datetime.date(2013, 6, 30), datetime.date(2014, 12, 31),
       datetime.date(2014, 9, 30), datetime.date(2014, 6, 30),
       datetime.date(2014, 3, 31), datetime.date(2013, 12, 31),
       datetime.date(2013, 9, 30), datetime.date(2015, 3, 31),
       datetime.date(2015, 6, 30), datetime.date(2015, 9, 30),
       datetime.date(2015, 12, 31), datetime.date(2016, 3, 31),
       datetime.date(2016, 6, 30), datetime.date(2016, 9, 30),
       datetime.date(2016, 12, 31), datetime.date(2017, 3, 31),
       datetime.date(2017, 6, 30), datetime.date(2017, 9, 30),
       datetime.date(2017, 12, 31), datetime.date(2018, 3, 31),
       datetime.date(2018, 6, 30), datetime.date(2018, 9, 30),
       datetime.date(2018, 12, 31), datetime.date(2019, 3, 31),
       datetime.date(2019, 6, 30), datetime.date(2019, 9, 30),
       datetime.date(2019, 12, 31), datetime.date(2020, 3, 31)],
      dtype=object)

In [87]:
# split years!!!!
# both form13f & prices：2013.6.30-2018.6.30 train; 2018.9.30-2020.3.31 test quarterly 每季test完把data加进train里？目标值是return？
# 构建因子，model selection# this is an expanding window